# e-Commerce Example

Here are sample data and queries for the example. First, let's create a new table `Store` with partition key `PK` and sort key `SK`. After that, generate sample data. PUtting sample data will be followed by the progress of requirements.

In [78]:
# import and get dynamodb resource
import csv
import random
from datetime import datetime, timedelta
import uuid
import boto3
import time
from pprint import pprint
from decimal import Decimal
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

dynamodb = boto3.resource('dynamodb')

In [88]:
# delete Store table if neccessary
try:
    dynamodb.Table('Store').delete()
except ClientError:
    pass

In [89]:
# create a table and wait until done
store = dynamodb.create_table(
    TableName='Store',
    AttributeDefinitions=[
        {
            'AttributeName': 'PK',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'SK',
            'AttributeType': 'S'
        }
    ],
    KeySchema=[
        {
            'AttributeName': 'PK',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'SK',
            'KeyType': 'RANGE'
        }
    ],
    BillingMode='PAY_PER_REQUEST'
)

store.wait_until_exists()

In [83]:
# generate sample user data
users = []

with open('data/users.csv', 'r') as f:
    for full_name, user_name, email in csv.reader(f):
        created_at = datetime.strftime(datetime.now() - timedelta(days=random.randint(0, 1825)), '%Y-%m-%d')
        users.append({
            'user_name': user_name,
            'full_name': full_name,
            'email': email,
            'created_at': created_at,
            'address': '..'
        })

pprint(users[0])

{'address': '..',
 'created_at': '2019-12-30',
 'email': 'trevor@gmail.com',
 'full_name': 'Trevor Sanderson',
 'user_name': 'trevor'}


In [84]:
# generate sample order and item data
orders = []

for user in users:
    for _ in range(random.randint(10, 100)):
        order = {
            'id': str(uuid.uuid4())[:8],
            'user_name': user['user_name'],
            'status': 'SHIPPED' if random.randint(0, 19) <= 18 else 'PLACED',
            'created_at': datetime.strftime(datetime.now() - timedelta(days=random.randint(0, 1825)), '%Y-%m-%d'),
            'address': '..'
        }
        order['status_id'] = order['status'] + '#' + order['id']
        if order['status'] == 'PLACED':
            order['placed_id'] = str(uuid.uuid4())[-4:]

        order['items'] = []

        for _ in range(random.randint(1, 10)):
            item = {
                'id': str(uuid.uuid4())[:8],
                'product_name': 'Product #' + str(random.randint(1, 1000)),
                'price': random.randint(1, 100000),
                'status': 'FILLED'
            }

            order['items'].append(item)
    
        orders.append(order)

pprint(orders[0])

{'address': '..',
 'created_at': '2016-10-24',
 'id': 'da63c048',
 'items': [{'id': 'd0995397',
            'price': 54211,
            'product_name': 'Product #262',
            'status': 'FILLED'},
           {'id': '69fdaf98',
            'price': 82912,
            'product_name': 'Product #45',
            'status': 'FILLED'},
           {'id': 'da24ef38',
            'price': 11584,
            'product_name': 'Product #261',
            'status': 'FILLED'},
           {'id': 'ef5f701f',
            'price': 22074,
            'product_name': 'Product #30',
            'status': 'FILLED'},
           {'id': '0cd4eabb',
            'price': 87381,
            'product_name': 'Product #678',
            'status': 'FILLED'},
           {'id': '153d584a',
            'price': 31112,
            'product_name': 'Product #875',
            'status': 'FILLED'},
           {'id': '6909b5f7',
            'price': 58412,
            'product_name': 'Product #255',
            'status': 'F

Here are the five access patterns described in the slide. We'll handle one by one.

1. Get user profile
2. Get orders for users
3. Get single order and order items
4. Get orders for user by status
5. Get open orders

## Access Pattern 1. Get User Profile

To do this, put user data into the table.

In [90]:
# put user data into the table
with store.batch_writer() as batch:
    for user in users:
        batch.put_item(Item={
            'PK': 'USER#' + user['user_name'],
            'SK': 'PROFILE#' + user['user_name'],
            'UserName': user['user_name'],
            'FullName': user['full_name'],
            'Email': user['email'],
            'CreatedAt': user['created_at']
        })

This is pretty straightforward. Just put user name into `PK` and `SK` with prefixes.

In [91]:
# get user profile
response = store.get_item(
    Key={'PK': 'USER#claire', 'SK': 'PROFILE#claire'}
)

pprint(response)

{'Item': {'CreatedAt': '2018-08-24',
          'Email': 'claire@gmail.com',
          'FullName': 'Claire Hardacre',
          'PK': 'USER#claire',
          'SK': 'PROFILE#claire',
          'UserName': 'claire'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '188',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Tue, 24 Nov 2020 01:45:46 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '3552840794',
                                      'x-amzn-requestid': 'SG1AB26D02DFIJ37MSVFSHLGD3VV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'SG1AB26D02DFIJ37MSVFSHLGD3VV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


## Access Pattern 2. Get Orders for Users

To do this, put order data into the table.

In [92]:
# put order data into the table
with store.batch_writer() as batch:
    for order in orders:
        batch.put_item(Item={
            'PK': 'USER#' + order['user_name'],
            'SK': 'ORDER#' + order['id'],
            'UserName': order['user_name'],
            'OrderId': order['id'],
            'Status': order['status'],
            'CreatedAt': order['created_at'],
            'Address': order['address']
        })

To get all orders of the user, put user name into `PK` with prefix and order prefix into `SK`. For simplicity, we limit up to 5 items here.

In [93]:
# get orders of the user
response = store.query(
    KeyConditionExpression=Key('PK').eq('USER#claire') & Key('SK').begins_with('ORDER#'),
    Limit=5
)

pprint(response)

{'Count': 5,
 'Items': [{'Address': '..',
            'CreatedAt': '2015-12-27',
            'OrderId': '05ad78d3',
            'PK': 'USER#claire',
            'SK': 'ORDER#05ad78d3',
            'Status': 'SHIPPED',
            'UserName': 'claire'},
           {'Address': '..',
            'CreatedAt': '2017-09-25',
            'OrderId': '09480e65',
            'PK': 'USER#claire',
            'SK': 'ORDER#09480e65',
            'Status': 'SHIPPED',
            'UserName': 'claire'},
           {'Address': '..',
            'CreatedAt': '2020-07-12',
            'OrderId': '12005c57',
            'PK': 'USER#claire',
            'SK': 'ORDER#12005c57',
            'Status': 'SHIPPED',
            'UserName': 'claire'},
           {'Address': '..',
            'CreatedAt': '2017-11-17',
            'OrderId': '140ac233',
            'PK': 'USER#claire',
            'SK': 'ORDER#140ac233',
            'Status': 'SHIPPED',
            'UserName': 'claire'},
           {'Address': '..'

## Access Pattern 3. Get single order and order items

To do this, put item data into the table. We also need to create the first index, which is inverted index (swapping `PK` and `SK`).

In [94]:
# put item data into the table
with store.batch_writer() as batch:
    for order in orders:
        for item in order['items']:
            batch.put_item(Item={
                'PK': 'ITEM#' + item['id'],
                'SK': 'ORDER#' + order['id'],
                'ItemId': item['id'],
                'OrderId': order['id'],
                'ProductName': item['product_name'],
                'Price': item['price'],
                'Status': item['status']
            })

In [95]:
# create an index, which is inverted index
store = store.update(
    AttributeDefinitions=[
        {'AttributeName': 'PK', 'AttributeType': 'S'},
        {'AttributeName': 'SK', 'AttributeType': 'S'}
    ],
    GlobalSecondaryIndexUpdates=[
        {
            'Create': {
                'IndexName': 'GSI-1',
                'KeySchema': [
                    {'AttributeName': 'SK', 'KeyType': 'HASH'},
                    {'AttributeName': 'PK', 'KeyType': 'RANGE'}
                ],
                'Projection': {
                    'ProjectionType': 'ALL',
                }
            }
        }
    ]
)

In [96]:
# wait until the index is created
gsi_status = 'N/A'

while gsi_status != 'ACTIVE':
    store.reload()
    
    for index in store.global_secondary_indexes:
        if index['IndexName'] == 'GSI-1':
            gsi_status = index['IndexStatus']
            print(datetime.now(), gsi_status)
    
    time.sleep(10)

2020-11-24 01:51:05.060973 CREATING
2020-11-24 01:51:15.078696 CREATING
2020-11-24 01:51:25.099620 CREATING
2020-11-24 01:51:35.120271 ACTIVE


Now the index `GSI-1`'s partition key `SK` has all the information of orders. Search an order on this index by putting order id into `SK`. The result will have the order's items and the orderer, which looks like a joined result.

In [98]:
# get an order and its items
response = store.query(
    IndexName='GSI-1',
    KeyConditionExpression=Key('SK').eq('ORDER#05ad78d3')
)

pprint(response)

{'Count': 9,
 'Items': [{'ItemId': '0797e0bb',
            'OrderId': '05ad78d3',
            'PK': 'ITEM#0797e0bb',
            'Price': Decimal('95876'),
            'ProductName': 'Product #917',
            'SK': 'ORDER#05ad78d3',
            'Status': 'FILLED'},
           {'ItemId': '499c2225',
            'OrderId': '05ad78d3',
            'PK': 'ITEM#499c2225',
            'Price': Decimal('28819'),
            'ProductName': 'Product #871',
            'SK': 'ORDER#05ad78d3',
            'Status': 'FILLED'},
           {'ItemId': '50205a03',
            'OrderId': '05ad78d3',
            'PK': 'ITEM#50205a03',
            'Price': Decimal('31044'),
            'ProductName': 'Product #268',
            'SK': 'ORDER#05ad78d3',
            'Status': 'FILLED'},
           {'ItemId': '6781a53f',
            'OrderId': '05ad78d3',
            'PK': 'ITEM#6781a53f',
            'Price': Decimal('68506'),
            'ProductName': 'Product #360',
            'SK': 'ORDER#05ad78d3',


## Access Pattern 4. Get orders for user by status

Firstly, getting orders with user and status filters is not efficient with the current table model. Here is the reason why we should do something more.

In [99]:
# get orders for user by status with the current table model
response = store.query(
    KeyConditionExpression=Key('PK').eq('USER#claire'),
    FilterExpression=Attr('Status').eq('PLACED'),
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Store'},
 'Count': 4,
 'Items': [{'Address': '..',
            'CreatedAt': '2016-07-14',
            'OrderId': '2bd89439',
            'PK': 'USER#claire',
            'SK': 'ORDER#2bd89439',
            'Status': 'PLACED',
            'UserName': 'claire'},
           {'Address': '..',
            'CreatedAt': '2019-06-02',
            'OrderId': '69130e3f',
            'PK': 'USER#claire',
            'SK': 'ORDER#69130e3f',
            'Status': 'PLACED',
            'UserName': 'claire'},
           {'Address': '..',
            'CreatedAt': '2019-04-14',
            'OrderId': '90a03066',
            'PK': 'USER#claire',
            'SK': 'ORDER#90a03066',
            'Status': 'PLACED',
            'UserName': 'claire'},
           {'Address': '..',
            'CreatedAt': '2017-02-23',
            'OrderId': 'fd214345',
            'PK': 'USER

Compare `Count` and `ScannedCount` values. The result means that DynamoDB retrieved `ScannedCount` items and filtered out most of the items in memory and returned `Count` items to us. Basically pricing depends on `ScannedCount`, not `Count`. Check `CapacityUnits` as well.

To enhance the cost efficiency and performance, we need to put `OrderStatusId` attribute in order items. After that, create an index on the new attribute.

In [100]:
# re-put order data with order_status_id attribute
with store.batch_writer() as batch:
    for order in orders:
        batch.put_item(Item={
            'PK': 'USER#' + order['user_name'],
            'SK': 'ORDER#' + order['id'],
            'UserName': order['user_name'],
            'OrderId': order['id'],
            'Status': order['status'],
            'CreatedAt': order['created_at'],
            'Address': order['address'],
            'OrderStatusId': order['status_id']
        })

In [101]:
# create an index on PK and OrderStatusId
store = store.update(
    AttributeDefinitions=[
        {'AttributeName': 'PK', 'AttributeType': 'S'},
        {'AttributeName': 'OrderStatusId', 'AttributeType': 'S'}
    ],
    GlobalSecondaryIndexUpdates=[
        {
            'Create': {
                'IndexName': 'GSI-2',
                'KeySchema': [
                    {'AttributeName': 'PK', 'KeyType': 'HASH'},
                    {'AttributeName': 'OrderStatusId', 'KeyType': 'RANGE'}
                ],
                'Projection': {
                    'ProjectionType': 'ALL',
                }
            }
        }
    ]
)

In [102]:
# wait until the index is created
gsi_status = 'N/A'

while gsi_status != 'ACTIVE':
    store.reload()
    
    for index in store.global_secondary_indexes:
        if index['IndexName'] == 'GSI-2':
            gsi_status = index['IndexStatus']
            print(datetime.now(), gsi_status)
    
    time.sleep(10)

2020-11-24 02:00:43.204401 CREATING
2020-11-24 02:00:53.225176 CREATING
2020-11-24 02:01:03.245049 CREATING
2020-11-24 02:01:13.266056 CREATING
2020-11-24 02:01:23.285529 CREATING
2020-11-24 02:01:33.301077 CREATING
2020-11-24 02:01:43.315000 CREATING
2020-11-24 02:01:53.334203 ACTIVE


Now we query on the new index. We see `Count`, `ScannedCount` and `CapacityUnits` and compare to the original result.

In [103]:
# get orders for user by status with the new index
response = store.query(
    IndexName='GSI-2',
    KeyConditionExpression=Key('PK').eq('USER#claire') & Key('OrderStatusId').begins_with('PLACED#'),
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 0.5,
                      'GlobalSecondaryIndexes': {'GSI-2': {'CapacityUnits': 0.5}},
                      'Table': {'CapacityUnits': 0.0},
                      'TableName': 'Store'},
 'Count': 4,
 'Items': [{'Address': '..',
            'CreatedAt': '2016-07-14',
            'OrderId': '2bd89439',
            'OrderStatusId': 'PLACED#2bd89439',
            'PK': 'USER#claire',
            'SK': 'ORDER#2bd89439',
            'Status': 'PLACED',
            'UserName': 'claire'},
           {'Address': '..',
            'CreatedAt': '2019-06-02',
            'OrderId': '69130e3f',
            'OrderStatusId': 'PLACED#69130e3f',
            'PK': 'USER#claire',
            'SK': 'ORDER#69130e3f',
            'Status': 'PLACED',
            'UserName': 'claire'},
           {'Address': '..',
            'CreatedAt': '2019-04-14',
            'OrderId': '90a03066',
            'OrderStatusId': 'PLACED#90a03066',
            'PK': 'USER#claire',
  

## Access Pattern 5. Get Open Orders

It would be important for admins to check open orders (order status is placed) to do the post-processing. With the current table model, the only way to get data is full-scan over the table as follows.

In [104]:
# scan all data having Status as PLACED
store.scan(
    FilterExpression=Attr('Status').eq('PLACED'),
    ReturnConsumedCapacity='INDEXES'
)

{'Items': [{'Address': '..',
   'SK': 'ORDER#1b73c0ea',
   'Status': 'PLACED',
   'PK': 'USER#alexander',
   'UserName': 'alexander',
   'OrderId': '1b73c0ea',
   'CreatedAt': '2019-06-19',
   'OrderStatusId': 'PLACED#1b73c0ea'},
  {'Address': '..',
   'SK': 'ORDER#2b357734',
   'Status': 'PLACED',
   'PK': 'USER#alexander',
   'UserName': 'alexander',
   'OrderId': '2b357734',
   'CreatedAt': '2019-04-04',
   'OrderStatusId': 'PLACED#2b357734'},
  {'Address': '..',
   'SK': 'ORDER#2fff1666',
   'Status': 'PLACED',
   'PK': 'USER#alexander',
   'UserName': 'alexander',
   'OrderId': '2fff1666',
   'CreatedAt': '2018-05-01',
   'OrderStatusId': 'PLACED#2fff1666'},
  {'Address': '..',
   'SK': 'ORDER#6b8f7c2b',
   'Status': 'PLACED',
   'PK': 'USER#alexander',
   'UserName': 'alexander',
   'OrderId': '6b8f7c2b',
   'CreatedAt': '2019-11-22',
   'OrderStatusId': 'PLACED#6b8f7c2b'},
  {'Address': '..',
   'SK': 'ORDER#7920dbe1',
   'Status': 'PLACED',
   'PK': 'USER#alexander',
   'UserNa

See `Count`, `ScannedCount` and `CapacityUnits`. Obviously, this is time consuming and cost ineffective. To do it more elegantly, we add `PlacedId` attribute on the open orders and create a sparse index on the attribute.

In [105]:
# re-put open order data with placed_id attribute
with store.batch_writer() as batch:
    for order in orders:
        if 'placed_id' in order:
            batch.put_item(Item={
                'PK': 'USER#' + order['user_name'],
                'SK': 'ORDER#' + order['id'],
                'UserName': order['user_name'],
                'OrderId': order['id'],
                'Status': order['status'],
                'CreatedAt': order['created_at'],
                'Address': order['address'],
                'OrderStatusId': order['status_id'],
                'PlacedId': order['placed_id']
            })

In [106]:
# create an index on PlacedId
store = store.update(
    AttributeDefinitions=[
        {'AttributeName': 'PlacedId', 'AttributeType': 'S'}
    ],
    GlobalSecondaryIndexUpdates=[
        {
            'Create': {
                'IndexName': 'GSI-3',
                'KeySchema': [
                    {'AttributeName': 'PlacedId', 'KeyType': 'HASH'}
                ],
                'Projection': {
                    'ProjectionType': 'ALL',
                }
            }
        }
    ]
)

In [107]:
# wait until the index is created
gsi_status = 'N/A'

while gsi_status != 'ACTIVE':
    store.reload()
    
    for index in store.global_secondary_indexes:
        if index['IndexName'] == 'GSI-3':
            gsi_status = index['IndexStatus']
            print(datetime.now(), gsi_status)
    
    time.sleep(10)

2020-11-24 02:16:18.388213 CREATING
2020-11-24 02:16:28.407862 CREATING
2020-11-24 02:16:38.427418 CREATING
2020-11-24 02:16:48.446254 CREATING
2020-11-24 02:16:58.465493 CREATING
2020-11-24 02:17:08.485788 CREATING
2020-11-24 02:17:18.505318 CREATING
2020-11-24 02:17:28.525032 ACTIVE


We're ready to do it better. Query on the new index and check `Count`, `ScannedCount` and `CapacityUnits` if it is done well.

In [108]:
# get open orders with the new index
response = store.scan(
    IndexName='GSI-3',
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 2.0,
                      'GlobalSecondaryIndexes': {'GSI-3': {'CapacityUnits': 2.0}},
                      'Table': {'CapacityUnits': 0.0},
                      'TableName': 'Store'},
 'Count': 25,
 'Items': [{'Address': '..',
            'CreatedAt': '2018-01-02',
            'OrderId': 'bc16994b',
            'OrderStatusId': 'PLACED#bc16994b',
            'PK': 'USER#alexandra',
            'PlacedId': 'ce67',
            'SK': 'ORDER#bc16994b',
            'Status': 'PLACED',
            'UserName': 'alexandra'},
           {'Address': '..',
            'CreatedAt': '2019-08-07',
            'OrderId': 'eeb41d51',
            'OrderStatusId': 'PLACED#eeb41d51',
            'PK': 'USER#caroline',
            'PlacedId': 'ee86',
            'SK': 'ORDER#eeb41d51',
            'Status': 'PLACED',
            'UserName': 'caroline'},
           {'Address': '..',
            'CreatedAt': '2020-06-30',
            'OrderId': '6d9e485d',
        